In [ ]:
import webvtt
from os import listdir
from os.path import isfile, join
import pandas as pd
from pydub import AudioSegment
import os
import shutil
import natsort 
import fileinput
import re

In [ ]:
rootdir="../Alugha/pre-data"
backup = "../Alugha/backup" # place holder for backup data


In [ ]:
"""
1. Alugha

step 1: Audio convertion
step 2: Deleting original file
step 3: Extracting segment information from .vtt filr and spliting the audio based on segment information.

"""

# (a) Audio convertion: -wav, 16K Hz, mono channel   
if True:
    for Dir in natsort.natsorted(os.listdir(rootdir)):
        print(Dir)        
        length = len(os.listdir(rootdir+"/"+Dir))
        print(length)
        if(length > 2):
            print("file already converted")
        else:
            for file in os.listdir(rootdir+"/"+Dir): 
                if(file.endswith(".mp4")):
                    actual_filename = file[:-4]+"_C"
                    a = os.system('ffmpeg -i "{}" -acodec pcm_s16le -ac 1 -ar 16000 "{}/{}".wav'.format(rootdir+"/"+Dir+"/"+file, rootdir+"/"+Dir+"/", actual_filename))
                    if(a==256):
                        print("Did not get converted")
#(b) Deleting original file
if True: 
    for Dir in natsort.natsorted(os.listdir(rootdir)):
            length = len(os.listdir(rootdir+"/"+Dir))
            if(length > 2):
                for file in os.listdir(rootdir+"/"+Dir):
                    if(file.endswith(".wav") and file.split('_')[-1]!="C.wav"):
                        os.remove(rootdir+"/"+Dir+"/"+file) 
                    if(file.endswith(".mp4")):
                        os.remove(rootdir+"/"+Dir+"/"+file)
            if(length == 2):
                print(" is being removed")                
                shutil.move(rootdir+"/"+Dir, backup)

                
# (c) creating chunks and saving in the same folder
if True:
    for Dir in os.listdir(rootdir):
        df  = pd.DataFrame(columns = ['A', 'B', 'C'])
        for file in os.listdir(rootdir+"/"+Dir):
            if(file.endswith(".vtt")):                
                for caption in webvtt.read(rootdir+"/"+Dir+"/"+file):
                    df2 = pd.DataFrame([[caption.start,caption.end,caption.text]], columns=list('ABC'))
                    df = df.append(df2)
                df = df.rename(index=str, columns={"A": "start", "B": "end","C" : "text"})
        for file in os.listdir(rootdir+"/"+Dir):
            if(file.endswith(".wav") and file.split('_')[-1]=="C.wav"):                
                myaudio = AudioSegment.from_file(rootdir+"/"+Dir+"/"+file, "wav")
                for i in range(0, len(df)):
                    start_time = df['start'].iloc[i]
                    end_time = df['end'].iloc[i]
                    # -----------------------------START-TIME-----------------------------------
                    s_hours, s_minutes, s_seconds = (["0", "0"] + start_time.split(":"))[-3:]
                    s_hours = int(s_hours)
                    s_minutes = int(s_minutes)
                    s_seconds = float(s_seconds)
                    s_millisecond = int(3600000 * s_hours + 60000 * s_minutes + 1000 * s_seconds)
                    START = s_millisecond
                    #-------------------------------END-TIME---------------------------------------
                    e_hours, e_minutes, e_seconds = (["0", "0"] + end_time.split(":"))[-3:]
                    e_hours = int(e_hours)
                    e_minutes = int(e_minutes)
                    e_seconds = float(e_seconds)
                    e_miliseconds = int(3600000 * e_hours + 60000 * e_minutes + 1000 * e_seconds)
                    END = e_miliseconds
                    #--------------------------------------------------------------------------
                    chunk = myaudio[START:END]
                    chunk_name = file.split('.')[-2]+"_"+str(i)+".flac"
                    if not os.path.exists(rootdir+"/"+Dir+"/"+chunk_name):
                        print("exporting", chunk_name)
                        chunk.export(rootdir+"/"+Dir+"/"+chunk_name, format="flac")
                if not os.path.exists(backup+"/"+file):
                    shutil.move(rootdir+"/"+Dir+"/"+file, backup)
                    print("Backing up")
                else:
                    os.remove(rootdir+"/"+Dir+"/"+file)
                    print("File already exist. So just removing")
        #----------------------------------------------txt--------------------------
        txt_name = ""        
        for file in natsort.natsorted(os.listdir(rootdir+"/"+Dir)):
            if(file.endswith(".vtt")):
                txt_name = file.split(".")[-2]
        with open(rootdir+"/"+Dir+"/"+txt_name+"_C.trans.txt", "w") as txt_file:
            i=0
            for file in natsort.natsorted(os.listdir(rootdir+"/"+Dir)):                
                if(file.endswith(".flac")):
                    wav_file_name = file.split(".")[-2]
                    txt_file.write(wav_file_name+" "+df['text'].iloc[i].upper()+"\n")
                    i = i+1
        for file in natsort.natsorted(os.listdir(rootdir+"/"+Dir)):
            if(file.endswith(".vtt")):
                if not os.path.exists(backup+"/"+file):
                    shutil.move(rootdir+"/"+Dir+"/"+file, backup)
                else:
                    os.remove(rootdir+"/"+Dir+"/"+file)
                    print("File already exist. So just removing")